In [ ]:
GSE_Train = "GSE63990"
target = "infection_status" #目标存的列。暂时还都不用改。

In [ ]:
# 各种库导入
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
import warnings
import GEOparse
from sklearn.metrics import roc_curve, auc, precision_recall_curve, confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LinearRegression, RidgeClassifier, Ridge, Lasso, LassoCV, LogisticRegression
from sklearn.svm import SVC, LinearSVC, NuSVC
warnings.filterwarnings('ignore')

In [ ]:
# 数据导入 及 预览数据. 输出末2行
gse = GEOparse.get_GEO(geo=GSE_Train, destdir="./datasets", silent=True)
gpls = gse.metadata['platform_id']
gpl = GEOparse.get_GEO(geo=gpls[0], destdir="./datasets", silent=True)
gse_csv = pd.read_csv('./datasets/' + GSE_Train + '.csv')
gse_csv.tail(2) # 预览数据

In [ ]:
# 选择的模型
up_feature =  ['ISG15', 'MX1', 'CD74']
up_weight =  [11,  8, 24]
up_times = 84694.59808314576
up_intercept = -2.8716859
up_model = LogisticRegression()
down_feature =  ['CD177', 'IL1R2', 'S100A12', 'MMP9']
down_weight = [14, 11, 24,  1]
down_times = 151481.89222583472
down_model = LogisticRegression()
down_intercept = -2.84741179

# 实际输入的模型
feature =  up_feature + down_feature
print('feature:', feature)
weight = up_weight + (np.array(down_weight) * -1).tolist()
print('weight:', weight)

In [ ]:
# 仅选择特征和最终结果. 输出末2行
ID = []
for i in range(len(feature)):
    ID.append(gpl.table[gpl.table['Gene Symbol'] == feature[i]]['ID'].values[0])
ID.append(target)
ID = np.array(ID).reshape(-1)
ID.astype('str')
testdata = gse_csv[ID]
# 把testdata的columns的名字改成feature
testdata.columns = feature + ["target"]
testdata.tail(2)

In [ ]:
# 计算每一种基因在不同感染情况下的表达量的平均值
testdata_mean = testdata.groupby("target").mean()
testdata_mean

In [ ]:
# 对平均值都乘以权重
testdata_mean_weight = testdata_mean * weight
testdata_mean_weight

In [ ]:
# 后面就是画图的部分了。 具体可以看 https://seaborn.pydata.org/tutorial.html
# 直接问ai也可以。 比如 https://www.phind.com/
# https://seaborn.pydata.org/tutorial/axis_grids.html 比如这个下面的几个例子都可以参考
fig, ax = plt.subplots(1,2, figsize=(20, 10)) # 画布大小横为20，纵为10，一共1行2列


In [ ]:
data1=testdata_mean.loc[['bacterial','viral']]
data2=testdata_mean_weight.loc[['bacterial','viral']]
data1.plot.bar(stacked=True,width=0.4)
data2.plot.bar(stacked=True,width=0.4,bottom=0)
plt.title('')
plt.show()